# Introduction

Nous allons dans ce notebook ecrire les differentes formules/fonctions utilisees dans Power BI, afin de creer de nouvelles colonnes par table dans l'onglet Data View, et des indicateurs(mesures) qui nous permettront d'avoir une meilleure vision globale de l'activite du supermarche.

NOTE: ce notebook n'est pas a executer

# Ajout de colonnes

## Table Calendrier

In [ ]:
# Creons une colonne weekend (samedi ou dimanche) pour classer les jours OUI/NON
weekend = if(or('calendrier'[Day Name]= 'Saturday', 'calendrier'[Day Name]= 'Sunday' ), 'O','N')

In [ ]:
# Creons une colonne Fin de mois
fin_de_mois = EOMONTH('calendrier'[date], 0)

## Table Clients

In [ ]:
#Creons une colonne pour connaitre l'age actuel (donnees datant de 1998!!!!, nous allons rajeunir les clients de 20 ans) 
age = DATEDIFF(clients['date_naissance'], TODAY(), YEAR) - 20

In [ ]:
# Creons une colonne priorite pour categoriser les clients
priorite = IF(clients[carte_fidelite]= "Golden" && clients[proprietaire]= "Y", "Haute", "Normale")

In [ ]:
# Creons une colonne pour recuperer les 3 premieres lettres du pays
pays_court_client = UPPER(LEFT(clients[pays_client], 3))

## Table Produits

In [ ]:
# Creons une colonne afin de classer les produits par prix (si prix de vente > 3$ => Haute, > 1$ => Moyenne, reste => Basse)
classe_produit = IF(produits[prix_vente_produit] > 3, "Haute", IF(produits[prix_vente_produit] > 1, "Moyenne", "Basse"))

## Table Magasins

In [ ]:
# Creons une colonne pour savoir depuis combien d'annees a ete fait le dernier remodeling
annees_dernier_remodeling = DATEDIFF(magasins[date_remodeling_magasin], TODAY(), YEAR) - 20

# Ajout des mesures

## Tables Transactions et Retours

In [ ]:
# Creons pour chaque table une mesure nous permettant de connaitre la quantite vendue, et la quantite retournee
quantite_retournee = SUM(retours[quantite])
quantite_vendue = SUM(transactions[quantite])

In [ ]:
# Mesure pour savoir combien a t on eu des transactions et de retours au total
total_transactions = COUNTROWS(transactions)
total_retours = COUNTROWS(retours)

In [ ]:
# Mesure pour savoir le taux de retour d'articles
taux_retour = DIVIDE([quantite_retournee],[quantite_vendue], "Pas de vente")

In [ ]:
# Mesure pour connaitre le nombre de transactions faites le weekend
transactions_weekend = CALCULATE(transactions[total_transactions], calendrier[weekend] = "O")

In [ ]:
# Taux de transactions faites le weeekend
%_transactions_weekend = DIVIDE(transactions[transactions_weekend], [total_transactions], "Pas de Ventes")

In [ ]:
# Recette totale basee sur le nbre de transactions et le prix de vente du produit
recette_totale = SUMX(transactions, transactions[quantite] * RELATED(produits[prix_vente_produit]))

In [ ]:
# Cout total base sur le nbre de transactions et le cout du produit
cout_total = sumx(transactions, transactions[quantite] * RELATED(produits[cout_produit]))

In [ ]:
# Mesure pour connaitre le benefice/profit
profit = [recette_totale] - [cout_total]

In [ ]:
# Marge du profit en %
marge_profit = DIVIDE([profit],[recette_totale])

In [ ]:
# Mesure recettes Year-to-Date pour savoir comment les recettes ont evolue au fil des annees
recette_YTD = CALCULATE([recette_totale], DATESYTD(calendrier[date]))

In [ ]:
# Mesures pour avoir un apercu des recettes, retours, transactions, benefice par rapport au mois precedent
transactions_dernier_mois = CALCULATE(transactions[total_transactions], DATEADD(calendrier[date], -1, MONTH))
profit_dernier_mois = CALCULATE([profit], DATEADD(calendrier[date], -1, MONTH)
recette_dernier_mois = CALCULATE([recette_totale], DATEADD(calendrier[date], -1, MONTH))
retours_dernier_mois = CALCULATE(retours[total_retours], DATEADD(calendrier[date], -1, MONTH))
objectif_recette = [recette_dernier_mois] * 1.05 # gagner 5% par rapport au dernier mois

## Table Produits

In [ ]:
# Savoir le nombre unique de produits
produits_uniques = DISTINCTCOUNT(produits[nom_produit])